<a href="https://colab.research.google.com/github/snowysunny/Colaboratory/blob/master/tensorflow1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 安装

In [0]:
# 不需要进行安装
#! pip install tensorflow-gpu

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6],[7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


# Tensorflow 基础

In [4]:
# Tensorflow 1 + 1
import tensorflow as tf
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [6]:
# 数值相加
a = tf.constant(1)
b = tf.constant(1)
c = tf.add(a, b)
print(c)

tf.Tensor(2, shape=(), dtype=int32)


In [7]:
# 矩阵相乘
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6],[7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


In [10]:
x = tf.get_variable(name='x', shape=[1], initializer=tf.constant_initializer(3.))

# 在tf.GradientTape()的上下文内，所有计算步骤都会被记录以用于求导
with tf.GradientTape() as tape:
  y = tf.square(x)  # y = x^2
  
# 计算y关于x的导数
y_grad = tape.gradient(y, x)
print([y.numpy(), y_grad.numpy()])

[array([9.], dtype=float32), array([6.], dtype=float32)]


这里x是一个初始化为3的变量(Variable)，使用tf.get_variable()声明。与普通张量一样，变量同样具有形状(shape)和类型(dtype)属性，不过使用变量需要有一个初始化过程，可以通过在tf.get_variable()中指定initializer参数来指定所使用的的初始化器。

变量和普通张量的一个重要区别是其默认能够被Tensorflow的自动求导机制所求导，因此常常被用于定义机器学习模型的参数。tf.GradientTape()是一个自动求导的记录器，在其中变量和计算步骤都会被自动记录。

**使用tf.GradientTape()计算函数$L(w, b) = ||Xw+b-y||^2$在$w=(1,2)^T, b=1$时对$w, b$的偏导数。 其中$ X=\begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} , y = \begin{bmatrix} 1 \\ 2 \\ \end{bmatrix} $**

In [14]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.get_variable('w', shape=[2, 1], initializer=tf.constant_initializer([[1.], [2.]]))
b = tf.get_variable('b', shape=[1], initializer=tf.constant_initializer([1.]))
with tf.GradientTape() as tape:
  L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))     # tf.reduce_sum()操作代表对输入张量的所有元素求和，输出一个形状为空的纯量张量  ， tf.square()操作代表对输入张亮的每一个元素求平方，不改变张量形状

w_grad , b_grad = tape.gradient(L, [w, b])
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

[62.5, array([[35.],
       [50.]], dtype=float32), array([15.], dtype=float32)]


# 基础示例： 线性回归

In [0]:
import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017])
y_raw = np.array([12000, 14000, 15000, 16500, 17500])

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [45]:
# 输出参数
X = tf.constant(X)
y = tf.constant(y)
print(X)
print(y)

tf.Tensor([0.   0.25 0.5  0.75 1.  ], shape=(5,), dtype=float64)
tf.Tensor([0.         0.36363636 0.54545455 0.81818182 1.        ], shape=(5,), dtype=float64)


In [66]:
# 权值
w = tf.get_variable('w', shape=[], initializer=tf.zeros_initializer(), dtype=tf.float64)
b = tf.get_variable('b', shape=[], initializer=tf.zeros_initializer(), dtype=tf.float64)
variables = [w, b]
print(variables)

[<tf.Variable 'w:0' shape=() dtype=float64, numpy=0.0>, <tf.Variable 'b:0' shape=() dtype=float64, numpy=0.0>]


In [0]:
num_epoch = 10000
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3)

In [0]:
for e in range(num_epoch):
  # 使用tf.GradientTape()记录损失函数的梯度信息
  with tf.GradientTape() as tape:
    y_pred = w * X + b
    loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y))
  # Tensorflow 自动计算损失函数关于自变量（模型参数）的梯度
  grads = tape.gradient(loss, variables)
  
  # Tensorflow自动根据梯度更新参数
  optimizer.apply_gradients(grads_and_vars=zip(grads, variables))


In [69]:
print(w.numpy())
print(b.numpy())

0.9763702100237027
0.05756498006354141


>* 使用tape.gradient(ys, xs)自动计算梯度
>* 使用optimizer.apply_gradients(grads_and_vars)自动更新模型参数
>* 使用tf.train.GradientDescentOptimizer(learning_rate=1e-3)声明一个梯度下降优化器(Optimizer)，学习率为1e-3
